# Notes

- Graph üzerinde y'ler `g.ndata["label"]` olarak tutulabilir, weather feature'ları `g.ndata["feature"]` with shape `(B, n_features, n_loc, n_window)`

# DGL Basics

## Heterograph - Homograph

In [29]:
import torch 
# HETEROGRAPH
graph_data = {
   ('drug', 'interacts', 'drug'): (torch.tensor([0, 1]), torch.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (torch.tensor([0, 1]), torch.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (torch.tensor([1]), torch.tensor([2]))
}
g = dgl.heterograph(graph_data)
g

Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
      num_edges={('drug', 'interacts', 'drug'): 2, ('drug', 'interacts', 'gene'): 2, ('drug', 'treats', 'disease'): 1},
      metagraph=[('drug', 'drug', 'interacts'), ('drug', 'gene', 'interacts'), ('drug', 'disease', 'treats')])

In [30]:
# HOMOGRAPH
graph_data = {
   ('node_type', 'edge_type', 'node_type') : (torch.tensor([0, 1, 0, 1]), torch.tensor([1, 2, 2, 3])),
}
g = dgl.heterograph(graph_data)
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

# GNN-FiLM

In [16]:
import dgl
from dgl.data import PPIDataset

train_set = PPIDataset(mode="train")

In [15]:
g = train_set[0]
print(g)
print(g.ndata["_ID"].shape, g.ndata["_ID"])
print(g.ndata["feat"].shape, g.ndata["feat"])
print(g.ndata["label"].shape, g.ndata["label"])

Graph(num_nodes=1767, num_edges=34085,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(50,), dtype=torch.float32), 'label': Scheme(shape=(121,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})
torch.Size([1767]) tensor([   0,    1,    2,  ..., 1764, 1765, 1766])
torch.Size([1767, 50]) tensor([[-0.0855, -0.0884, -0.1128,  ..., -0.1399, -0.1494, -0.1481],
        [-0.0855, -0.0884, -0.1128,  ..., -0.1399, -0.1494, -0.1481],
        [-0.0855, -0.0884, -0.1128,  ..., -0.1399, -0.1494, -0.1481],
        ...,
        [-0.0855, -0.0884, -0.1128,  ..., -0.1399, -0.1494, -0.1481],
        [-0.0855, -0.0884, -0.1128,  ..., -0.1399, -0.1494, -0.1481],
        [-0.0855, -0.0884, -0.1128,  ..., -0.1399, -0.1494, -0.1481]])
torch.Size([1767, 121]) tensor([[1., 0., 0.,  ..., 1., 1., 0.],
        [1., 0., 0.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
    

In [19]:
i = 0
(train_set[i].nodes(), train_set[i].nodes())

(tensor([   0,    1,    2,  ..., 1764, 1765, 1766]),
 tensor([   0,    1,    2,  ..., 1764, 1765, 1766]))

In [ ]:
i = 0
g = dgl.heterograph({
    ('_N','_E','_N'): train_set[i].edges(),
    ('_N', 'self', '_N'): (train_set[i].nodes(), train_set[i].nodes())
})

In [ ]:
g.ndata['label'] = train_set[i].ndata['label']
g.ndata['feat'] = train_set[i].ndata['feat']
g.ndata['_ID'] = train_set[i].ndata['_ID']

In [64]:
train_set.graphs

[Graph(num_nodes=1767, num_edges=34085,
       ndata_schemes={'label': Scheme(shape=(121,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=1377, num_edges=31081,
       ndata_schemes={'label': Scheme(shape=(121,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=2263, num_edges=61907,
       ndata_schemes={'label': Scheme(shape=(121,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(50,), dtype=torch.float32)}
       edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}),
 Graph(num_nodes=2339, num_edges=67769,
       ndata_schemes={'label': Scheme(shape=(121,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme

In [62]:
train_set[i]

Graph(num_nodes=1767, num_edges=34085,
      ndata_schemes={'label': Scheme(shape=(121,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(50,), dtype=torch.float32)}
      edata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)})

In [60]:
from collections import namedtuple
BatchGraph = namedtuple('BatchGraph', ['graph', 'label'])

def batcher(device):
    def batcher_dev(batch):
        batch_graphs = dgl.batch(batch)
        return BatchGraph(graph=batch_graphs, label=batch_graphs.ndata['label'].to(device))
    return batcher_dev